In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None
import numpy as np
import json
from data_transfer_utility.application import DataTransferUtility
from xgboost import XGBClassifier
import shap
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from sklearn import metrics
from lifelines.utils import concordance_index
import random
import copy
from cryptography.fernet import Fernet
import pickle

import re
import os
import datetime
from datetime import date,timedelta
import matplotlib.pyplot as plt

today_date = date.today()

In [ ]:
import os
output_dir = f'data_backup/m2_mds_2024-05-31/inference/'
if output_dir.split('/')[-2] not in os.listdir('/'.join(output_dir.split('/')[:-2])):
    os.makedirs(output_dir)
text_file = open(output_dir+'output.txt','w')
# text_file = open(output_dir+'output.txt','a')

In [ ]:
from mds_features_model2 import *
from mds_labs_model2 import *

In [ ]:
REDSHIFT_CREDENTIALS = json.load(open('../secrets/dtu_redshift_credentials.json', 'r'))
dtu = DataTransferUtility(df_mode='pandas', **REDSHIFT_CREDENTIALS)

In [ ]:
# biomarker_schemas = {'Lung':'c3_gn360_202312_nsclc',
#                      'Breast':'c3_gn360_202309_breast',
#                      'Prostate':'c3_gn360_202309_prostate',
#                      'Bladder':'c3_gn360_202309_bladder',
#                      'Pancreas':'c3_gn360_202309_pancreatic',
#                      'Colorectal':'c3_gn360_202309_crc'}

biomarker_schemas= {'Bladder':'c3_gn360_202403_bladder',
                    'Breast':'c3_gn360_202403_breast',
                    'Colorectal':'c3_gn360_202403_crc',
                    'NSCLC':'c3_gn360_202403_nsclc',
                    'Ovarian':'c3_gn360_202403_ovarian',
                    'Pancreas':'c3_gn360_202403_pancreatic',
                    'Prostate':'c3_gn360_202403_prostate',
                    'SCLC':'c3_gn360_202403_sclc'}

In [ ]:
from datetime import date
import re
import time
import datetime
from datetime import timedelta

def _identify_following_month_first_date(month, year):
    if month == 12:
        cutoff_year = int(year) + 1
        cutoff_month = 1
    else:
        cutoff_year = year
        cutoff_month = month + 1

    cut_off_date = datetime.datetime(
        int(cutoff_year), cutoff_month, 1
    )

    return cut_off_date


def prepare_cutoff_date(schema_name):
    current_year = str(date.today()).split("-")[0]
    prev_year = str(int(str(date.today()).split("-")[0]) - 1)
    if re.search(current_year + r"(\d+)", schema_name):
        year = current_year
    elif re.search(prev_year + r"(\d+)", schema_name):
        year = prev_year
    month = int(re.search(year + r"(\d+)", schema_name).group(1))
    cut_off_date = _identify_following_month_first_date(month, year)

    return (cut_off_date - timedelta(days=1))


In [ ]:
def main(schema,index_date_table,lot_table_name, cohort):
    feature_dict = {}
    cohort_org = copy.deepcopy(cohort)
    if cohort in ['NSCLC','SCLC']:
        cohort='Lung'
    # cond_table = get_condition_table(schema, cohort)
    #     pat_ids = list(cond_table.unique())

    # index_date_df = get_index_table(index_date_table, cond_table)
    # index_date_df = get_index_table_wo_cond(index_date_table)
    index_date_df = get_mds_initial_dx_table(initial_dx_table)
    index_date_df = index_date_df.rename(columns={'diagnosis_date':'index_date'})
    index_date_df = index_date_df[index_date_df["index_date"].notnull()]
    print(f"Index_date : {index_date_df.shape}, unique patients : {index_date_df['chai_patient_id'].nunique()}")

    # cond_table = cond_table.merge(index_date_df[['chai_patient_id','index_date']],on=['chai_patient_id'],how='inner')
    cond_table = index_date_df[['chai_patient_id','index_date']]
    cond_table = cond_table[cond_table['index_date'].dt.year>2014]
    # cond_table = cond_table[cond_table['index_date']>cond_table['diagnosis_date']]
    cond_table = cond_table.drop_duplicates(subset=['chai_patient_id','index_date'])
    
    censored = get_censored_patients(schema, cond_table)
    cut_off_date = prepare_cutoff_date(schema)
    censored["cut_off_date"] = cut_off_date
    censored['last_available_date'] = censored.apply(lambda x:x['cut_off_date'] if (x['cut_off_date']<x['last_available_date']) else x['last_available_date'],axis=1)
    print(f"censored : {censored.shape}, unique patients : {censored['chai_patient_id'].nunique()}")

    cond_table =cond_table.merge(censored,on='chai_patient_id',how='left')
    cond_table['duration'] = cond_table.apply(lambda x:(x['last_available_date']-x['index_date']).days,axis=1)
    cond_table = cond_table[cond_table['duration']>0]
    print(f"condition : {cond_table.shape}, unique patients : {cond_table['chai_patient_id'].nunique()}")
    cond_table['random_point'] = cond_table.apply(
        lambda x: 0 if x['duration'] in [0, 1] else random.randint(1, x['duration'] - 1), axis=1)
    # cond_table['random_date'] = cond_table.apply(lambda x:x['index_date']+timedelta(days=x['random_point']),axis=1)
    cond_table['random_date'] = pd.Timestamp('2021-07-01')
    cond_table = cond_table[(cond_table['index_date']<=cond_table['random_date']) & (cond_table['last_available_date']>cond_table['random_date'])]


    lot_table = get_lot_table(lot_table_name, cond_table=cond_table)
    cond_table = cond_table.merge(lot_table,on='chai_patient_id',how='left')

    cond_table['lot_diff'] = cond_table.apply(lambda x:(x['regimen_end']-x['random_date']).days,axis=1)
    cond_table['lot_diff'] = cond_table['lot_diff'].apply(lambda x:np.nan if x<=0 else x)
    cond_table = cond_table.sort_values(by=['lot_diff'],ascending=True)
    cond_table = cond_table.drop_duplicates(subset=['chai_patient_id'],keep='first')
    cond_table['event'] = cond_table['lot_diff'].apply(lambda x:0 if pd.isna(x) else 1)
    cond_table['TTE'] = cond_table.apply(lambda x:x['lot_diff'] if (x['event']==1) else (x['last_available_date']-x['random_date']).days,axis=1)

    # cond_table['curation'] = cond_table['chai_patient_id'].apply(lambda x:random.randint(0,1))
    # print(cond_table['curation'].value_counts())
    
    cond_table.to_csv(f'{output_dir}cond_table_{cohort_org}_{today_date}_inference.csv')

    text_file.write(f"Cohort : {cohort_org}")
    text_file.write(f"Label value counts : \n{cond_table['event'].value_counts()}\n\n")
    text_file.write(f"Duration stats :\n")
    text_file.write(f"For event data points (event = 1) : \n{cond_table[cond_table['event']==1]['duration'].describe()}\n\n")
    text_file.write(f"For censor data points (event = 0) : \n{cond_table[cond_table['event']==0]['duration'].describe()}\n\n")
    text_file.write(f"TTE stats :\n")
    text_file.write(f"For event data points (event = 1) : \n{cond_table[cond_table['event']==1]['TTE'].describe()}\n\n")
    text_file.write(f"For censor data points (event = 0) : \n{cond_table[cond_table['event']==0]['TTE'].describe()}\n\n")
    cond_table[cond_table['event']==1]['duration'].hist(bins=100)
    plt.savefig(f'{output_dir}_duration_event_hist.png')
    plt.show()
    cond_table[cond_table['event']==0]['duration'].hist(bins=100)
    plt.savefig(f'{output_dir}_duration_censor_hist.png')
    plt.show()
    cond_table[cond_table['event']==1]['TTE'].hist(bins=100)
    plt.savefig(f'{output_dir}_tte_event_hist.png')
    plt.show()
    cond_table[cond_table['event']==0]['TTE'].hist(bins=100)
    plt.savefig(f'{output_dir}_tte_censor_hist.png')
    plt.show()


    # if cohort in biomarker_schemas:
    #     # biomarker_df = get_biomarker(biomarker_schemas[cohort],schema,cond_table,cohort)
    #     biomarker_df = get_all_biomarker(biomarker_schemas[cohort],schema,cond_table,cohort)
    #     biomarker_df = biomarker_df.drop('line_start',axis=1)
    #     feature_dict['biomarker'] = list(biomarker_df.columns)
    #     print(f"Biomarker : {biomarker_df.shape}, unique patients : {biomarker_df['chai_patient_id'].nunique()}")

    # imaging_df = get_imaging(schema, cond_table)
    # feature_dict['imaging'] = list(imaging_df.columns)
    # print(f"Imaging : {imaging_df.shape}, unique patients : {imaging_df['chai_patient_id'].nunique()}")
    
    # med_df = get_medication(schema, cond_table)
    # med_df = med_df.drop('line_start',axis=1)
    # feature_dict['Med'] = list(med_df.columns)
    # print(f"med : {med_df.shape}, unique patients : {med_df['chai_patient_id'].nunique()}")

    # conmed_df = get_conmed(schema, cond_table)
    # conmed_df = conmed_df.drop('line_start',axis=1)
    # feature_dict['ConMed'] = list(conmed_df.columns)
    # print(f"conmed : {conmed_df.shape}, unique patients : {conmed_df['chai_patient_id'].nunique()}")

    stage_df = get_stage(schema, cond_table, cohort)
    # stage_df = stage_df.drop(['line_start','stage_date'],axis=1)
    feature_dict['Staging'] = list(stage_df.columns)
    print(f"stage : {stage_df.shape}, unique patients : {stage_df['chai_patient_id'].nunique()}")

    # disease_statis_df = get_disease_status(schema, cond_table)
    # feature_dict['Disease Statis'] = list(disease_statis_df.columns)
    # print(f"disease_status : {disease_statis_df.shape}, unique patients : {disease_statis_df['chai_patient_id'].nunique()}")

    final_stat_df = get_labs(schema, cond_table)
    # final_stat_df = final_stat_df.drop('line_start',axis=1)
    feature_dict['Labs'] = list(final_stat_df.columns)
    print(f"lab : {final_stat_df.shape}, unique patients : {final_stat_df['chai_patient_id'].nunique()}")

    final_stat_df = cond_table[['chai_patient_id','TTE','event']].merge(final_stat_df,on=['chai_patient_id'],how='left')

    # final_stat_df = final_stat_df.merge(med_df,on=['chai_patient_id'],how='left')
    # final_stat_df = final_stat_df.merge(conmed_df,on=['chai_patient_id'],how='left')
    #     final_stat_df = final_stat_df.merge(comorb_df,on='chai_patient_id',how='left')
    #     final_stat_df = final_stat_df.merge(surgery_df,on=['chai_patient_id', 'line_start'],how='left')

    #     final_stat_df = final_stat_df.merge(demographics_df,on=['chai_patient_id', 'line_start'],how='left')
    #     final_stat_df = final_stat_df.merge(radiation_df,on=['chai_patient_id'],how='left')
    final_stat_df = final_stat_df.merge(stage_df,on=['chai_patient_id'],how='left')
    #     final_stat_df = final_stat_df.merge(tumor_exam_df,on=['chai_patient_id', 'line_start'],how='left')
    # final_stat_df = final_stat_df.merge(disease_statis_df,on=['chai_patient_id'],how='left')
    # if cohort in biomarker_schemas:
    #     final_stat_df = final_stat_df.merge(biomarker_df,on=['chai_patient_id'],how='left')
    # final_stat_df = final_stat_df.merge(imaging_df,on=['chai_patient_id'],how='left')
    #     final_stat_df = final_stat_df.merge(censored,on=['chai_patient_id','line_start'],how='left')
    #     final_stat_df['line_duration'] = final_stat_df.apply(lambda x: -x['line_duration'] if x['censored']=='yes' else x['line_duration'], axis=1)
    # final_stat_df['censored'] = final_stat_df.apply(lambda x: False if x['censored']==True else True, axis=1)

    return final_stat_df

In [ ]:
# text_file.close()

In [ ]:
# LOT_TABLES = dtu.s3_to_df('s3://c3-dev-model-factory/sai_vinod/sept_2023_pt360_delivery_unified_LOT_COT.json')['Patient360']['standard']
# def get_decoded_from_lot_table(lot_table):    
#     check = dtu.redshift_to_df(
#                             ''' SELECT version_of_model FROM {source_schema}'''.format(source_schema=lot_table))
#     secret_key = u'Kizqfoc8KapwPI4STeBfofaA_rV7nZNnbPSxLdT9KSM='
#     cipher_suite = Fernet(secret_key.encode('utf-8'))
#     a = check['version_of_model'].astype(str).values[1]  # check is LoT/CoT table dataframe
#     a = cipher_suite.decrypt((str.encode(a[2:-1])))
#     decoded = json.loads(a.decode("utf-8").replace("'", '"'))
#     schema = decoded['source_schema']
#     return decoded

In [ ]:
table_details = {'mds' : {'schema' : 'cdm_structured_202402',
                            'initial_dx_table' : 'model_factory_dev.initial_dx_date_cdm_structured_202402_v2_5_20240404_1034',
                            'lot_table_name' : 'line_of_therapy.mds_rbs_rwd360_cdm_structured_202402_lot_v6_7_20240404_1146'
                           }
                }

In [ ]:
%%time
dataframes = []
# patient_count = []
import time

for cohort in ['mds']:
    start = time.time() 
    print(cohort)
    
    schema = table_details[cohort]['schema']
    initial_dx_table = table_details[cohort]['initial_dx_table']
    lot_table_name = table_details[cohort]['lot_table_name']
    features = main(schema, initial_dx_table, lot_table_name, cohort)
    # patient_count.append(unique_patients)

    dataframes.append(features)
    print(time.time()-start)
# pickle.dump(dataframes, open('data/features_additional_lines_sept20.pkl', 'wb'))

In [ ]:
schema_dict

In [ ]:
count_df = pd.DataFrame({'cohort':['Pancreas', 'Melanoma', 'Colorectal', 'Prostate', 'Bladder', 'Breast', 'Gastricesophagus', 'HCC', 'Lung', 'Renal'],'pat_count':[1326, 344, 268, 844, 472, 1144, 869, 133, 5900, 689]})

In [ ]:
import matplotlib.pyplot as plt
count_df = count_df.sort_values(by='pat_count',ascending=True)
plt.barh(count_df['cohort'],count_df['pat_count'])

In [ ]:
total_pat = sum(count_df['pat_count'])
print(f'Total patients : {total_pat}')
count_df['percentage'] = (count_df['pat_count']*100)/(total_pat)
count_df

In [ ]:
cohort = 'Renal'
decoded = get_decoded_from_lot_table(LOT_TABLES[cohort]['schema']+'.'+ LOT_TABLES[cohort]['table'])
schema = decoded['source_schema']
lot_table_name = LOT_TABLES[cohort]['schema']+'.'+LOT_TABLES[cohort]['table']
print(schema)
print(lot_table_name)
print(cohort)

In [ ]:
features,unique_patients = main(schema, lot_table_name, cohort)

In [ ]:
# dataframe_list = pickle.load(open('data/features_sept20.pkl', 'rb'))

final_data = pd.concat(dataframes).drop_duplicates(subset=['chai_patient_id']).reset_index().drop('index',axis=1)

In [ ]:
final_data = copy.deepcopy(final_stat_df)
final_data.shape

In [ ]:
final_data.head()

In [ ]:
final_data['chai_patient_id'].nunique()

In [ ]:
final_data1 = copy.deepcopy(final_data)

In [ ]:
cohort_pat_list = list(final_data[['chai_patient_id','Complete therapeutic response_wind_0_45',
 'Partial therapeutic response_wind_0_45',
 'Stable_wind_0_45',
 'Tumor progression_wind_0_45',
 'Complete therapeutic response_wind_45_180',
 'Partial therapeutic response_wind_45_180',
 'Stable_wind_45_180',
 'Tumor progression_wind_45_180',
 'tp53_mutation','CT',
 'PET','MRI','Radioisotope']].dropna(how='all')['chai_patient_id'])
final_data= final_data[final_data['chai_patient_id'].isin(cohort_pat_list)]
final_data = final_data.reset_index().drop('index',axis=1)
final_data.shape

In [ ]:
final_data.head()

In [ ]:
# cohort_pat_list = list(final_data.drop(['TTE','event'],axis=1).dropna(how='all')['chai_patient_id'])
# final_data= final_data[final_data['chai_patient_id'].isin(cohort_pat_list)]
# final_data = final_data.reset_index().drop('index',axis=1)
# final_data.shape

In [ ]:
# from xgbse import XGBSEKaplanNeighbors
# from xgbse._kaplan_neighbors import DEFAULT_PARAMS
from xgbse.metrics import concordance_index
from xgbse import XGBSEDebiasedBCE
from xgbse.converters import convert_to_structured
import os
import time
# medication_df = pickle.load(open('data/medication_df.pkl', 'rb'))

# drug_list = pd.read_csv('data/cancer_drugs_public_V13_0.csv')
# chemo_drug_list = drug_list[drug_list['primary_category']=='Chemotherapy']
# chemo = []
# for col in medication_df.columns:
#     if col in chemo_drug_list['active_ingredient'].tolist():
#         print(col)
#         chemo.append(col)
# medication_df = medication_df[chemo + ['chai_patient_id', 'line_start']]

In [ ]:
final_data.shape

In [ ]:
# final_data1[['Complete therapeutic response_wind_0_45',
#  'Partial therapeutic response_wind_0_45',
#  'Stable_wind_0_45',
#  'Tumor progression_wind_0_45',
#  'Complete therapeutic response_wind_45_180',
#  'Partial therapeutic response_wind_45_180',
#  'Stable_wind_45_180',
#  'Tumor progression_wind_45_180']].dropna(how='all').shape

In [ ]:
# final_data[['Complete therapeutic response_wind_0_45',
#  'Partial therapeutic response_wind_0_45',
#  'Stable_wind_0_45',
#  'Tumor progression_wind_0_45',
#  'Complete therapeutic response_wind_45_180',
#  'Partial therapeutic response_wind_45_180',
#  'Stable_wind_45_180',
#  'Tumor progression_wind_45_180','tp53_mutation','CT',
#  'PET',
#  'MRI',
#  'Radioisotope']].dropna(how='all').shape

In [ ]:
# new_labs = pickle.load(open('mc/lab_features_sept_14_0.pkl', 'rb'))
# dataframe_list = pickle.load(open('data/features_sept20.pkl', 'rb'))
# final_data = pd.concat(dataframe_list).drop_duplicates().drop(columns = [np.nan], axis = 1)
# final_data = pd.concat([final_data, final_data1]).drop_duplicates()
# cohort_df = pd.get_dummies(final_data, columns = ['cohort', 'ethnicity', 'gender', 'race'], prefix = None)
# cohort_df = cohort_df.drop(np.nan, axis = 1)
# drop_these = []
# for col in list(cohort_df.columns):
#     for s in ['Mean', 'Median_', 'SD_', 'Minimum_', 'slope',
#               'Maximum_', 'Skewness_', 'Kurtosis_', '25th_Percentile_', '75th_Percentile_', 'Range_']:
#         if col.startswith(s):
#             drop_these.append(col)
# cohort_df = cohort_df.drop(columns = drop_these, axis = 1)
# cohort_df = cohort_df.merge(new_labs, on = ['chai_patient_id', 'line_start'], how = 'left')
# X = final_data.drop(['chai_patient_id','last_available_date','line_start','line_end','line_duration','random_point','line_number','target_duration',
#                      'censored', 'stage_date'],axis=1)
X = final_data.drop(['chai_patient_id','TTE','event'],axis=1)

# Y = cohort_df[['line_duration']]

regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=348)
# y_train['event'] = y_train.apply(lambda x: 1 if x['line_duration']>0 else 0, axis = 1)
# y_test['event'] = y_test.apply(lambda x: 1 if x['line_duration']>0 else 0, axis = 1)

# y_train_np = y_train.to_records(index=False)
# y_train_np = np.array(y_train_np, dtype = y_train_np.dtype.descr)

# y_test_np = y_test.to_records(index = False)
# y_test_np = np.array(y_test_np, dtype = y_test_np.dtype.descr)



# print("======================")
# # loop to show different scale results
# m = []
# for scale in [1.0]:
#     start = time.time()
#     DEFAULT_PARAMS['aft_loss_distribution_scale'] = scale
#     xgbse_model = XGBSEKaplanNeighbors(DEFAULT_PARAMS, n_neighbors=30)
#     xgbse_model.fit(X_train, y_train_np)
#     m.append(xgbse_model)
#     preds = xgbse_model.predict(X_test)
#     preds_train = xgbse_model.predict(X_train)
#     cind_test = concordance_index(y_test_np, preds)
#     cind_train = concordance_index(y_train_np, preds_train)
#     print(f"aft_loss_distribution_scale: {scale}")
#     print(f"C-index, train: {cind_train:.3f}")
#     print(f"C-index, test: {cind_test:.3f}")
#     #print(f"Average probability of survival: {avg_probs}")
#     print("----", time.time() - start)
# # pickle.dump(m, open('models/xgbse_models_nlp_sept20_additional_lines.pkl', 'wb'))

In [ ]:
# final_data['censored']=final_data['censored'].apply(lambda x:False if x==True else True)

In [ ]:
y = convert_to_structured((final_data['TTE']),final_data['event'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=938)

In [ ]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
# column_list = list(x_train.columns)
# len(column_list)

In [ ]:
# with open(f'data_backup/column_list_11_28_2023.pkl','wb') as f:
#     pickle.dump(column_list,f)

In [ ]:
PARAMS_XGB_AFT = {
    'objective': 'survival:aft',
    'eval_metric': 'aft-nloglik',
    'aft_loss_distribution': 'normal',
    'aft_loss_distribution_scale': 1.0,
    'tree_method': 'hist',
    'learning_rate': 0.005,
    'max_depth': 16,
    'booster':'dart',
    'subsample': 0.8,
    'min_child_weight': 30,
    'colsample_bynode':0.8
}

xgbse_model = XGBSEDebiasedBCE(PARAMS_XGB_AFT)
xgbse_model.fit(x_train,y_train,time_bins=np.array([60,90,120,150]))

In [ ]:
train_cindex_list = []
test_cindex_list = []
train_cindex_list.append(concordance_index(y_train, xgbse_model.predict(x_train)))
test_cindex_list.append(concordance_index(y_test, xgbse_model.predict(x_test)))

In [ ]:
print(f'Train c-index list : {train_cindex_list}')
print(f'Test c-index list: {test_cindex_list}')
print(f'Train c-index average : {np.mean(train_cindex_list)}')
print(f'Test c-index average: {np.mean(test_cindex_list)}')
text_file.write(f'Train c-index average : {np.mean(train_cindex_list)} \n')
text_file.write(f'Test c-index average: {np.mean(test_cindex_list)} \n')

In [ ]:
pred = []
pred.append(xgbse_model.predict(x_test))
    
event_prob_df = 1-pred[0]
# event_prob_df.to_csv(f'data_backup/all_feature_prob_{today_date}.csv')
text_file.write('Number of patients above 0.8 prediction probability\n')
slelected_pt_list = []
for col in event_prob_df.columns:
    selected_df = event_prob_df[event_prob_df[col]>=0.8]
    main_index = list(x_test.reset_index().iloc[list(selected_df.index)]['index'])
    slelected_pt_list.append(list(final_data.iloc[main_index]['chai_patient_id']))
    print(f'{col} : {selected_df.shape}')
    text_file.write(f'For window {col} : {selected_df.shape[0]}')

In [ ]:
import matplotlib.pyplot as plt
for i in [60,90,120,150]:
    print(f'window : {i}')
    event_prob_df[i].hist(bins=100)
    plt.savefig(f'{output_dir}predicted_prob_distribution_{i}.png')
    plt.show()

In [ ]:
selcted_patient_info = pd.DataFrame({'chai_patient_id':slelected_pt_list[3]})
window =[60,90,120,150]
for i in range(len(slelected_pt_list)):
    selcted_patient_info[window[i]] = selcted_patient_info['chai_patient_id'].apply(lambda x:True if x in slelected_pt_list[i] else False)

selcted_patient_info = selcted_patient_info.merge(final_data,on='chai_patient_id',how='inner') 

In [ ]:
# import pickle
# import joblib
# date = '02_13_2024'
# event_prob_df.to_csv(f'{output_dir}event_prob_df_{date}.csv')
# x_train.to_csv(f'{output_dir}x_train_{date}.csv')
# x_test.to_csv(f'{output_dir}x_test_{date}.csv')
# np.save(f'{output_dir}y_train_{date}.npy',y_train)
# np.save(f'{output_dir}y_test_{date}.npy',y_test)
# final_data.to_csv(f'{output_dir}final_data_{date}.csv')
# with open(f'{output_dir}model_{date}.pkl','wb') as f:
#     pickle.dump(xgbse_model,f)
# joblib.dump(xgbse_model,f'{output_dir}model_{date}.joblib')

In [ ]:
output_dir

In [ ]:
import pickle
import joblib
date = '05_31_2024'
# event_prob_df = pd.read_csv(f'{output_dir}event_prob_df_{date}.csv')
x_train = pd.read_csv(f'data_backup/m2_mds_2024-05-31/x_train_{date}.csv')
# x_test = pd.read_csv(f'{output_dir}x_test_{date}.csv')
# y_train = np.load(f'{output_dir}y_train_{date}.npy')
# y_test = np.load(f'{output_dir}y_test_{date}.npy')
# final_data = pd.read_csv(f'{output_dir}final_data_{date}.csv')

with open(f'data_backup/m2_mds_2024-05-31/model_{date}.pkl','rb') as f:
    xgbse_model = pickle.load(f) 

In [ ]:
x_train = x_train.drop('Unnamed: 0',axis=1)

In [ ]:
x_train.columns

In [ ]:
len(x_train.columns)

In [ ]:
len(['Delta_mean_ecog_0_45',
       'Delta_min_ecog_0_45', 'Delta_max_ecog_0_45', 'Mean_ecog_0_45',
       'Median_ecog_0_45', 'SD_ecog_0_45', 'Minimum_ecog_0_45',
       'Maximum_ecog_0_45', 'Skewness_ecog_0_45', 'Kurtosis_ecog_0_45',
       '25th_Percentile_ecog_0_45', '75th_Percentile_ecog_0_45',
       'Range_ecog_0_45', 'slope_ecog_0_45', 'Delta_mean_ecog_45_180',
       'Delta_min_ecog_45_180', 'Delta_max_ecog_45_180', 'Mean_ecog_45_180',
       'Median_ecog_45_180', 'SD_ecog_45_180', 'Minimum_ecog_45_180',
       'Maximum_ecog_45_180', 'Skewness_ecog_45_180', 'Kurtosis_ecog_45_180',
       '25th_Percentile_ecog_45_180', '75th_Percentile_ecog_45_180',
       'Range_ecog_45_180', 'slope_ecog_45_180', 'durvalumab', 'afatinib',
       'osimertinib', 'pembrolizumab', 'adagrasib', 'docetaxel', 'ceritinib',
       'gemcitabine', 'atezolizumab', 'etoposide', 'pemetrexed', 'ipilimumab',
       'crizotinib', 'carboplatin', 'sotorasib', 'gefitinib', 'alectinib',
       'ramucirumab', 'amivantamab', 'necitumumab', 'brigatinib', 'lorlatinib',
       'bevacizumab', 'dacomitinib', 'erlotinib', 'cisplatin', 'paclitaxel',
       'mobocertinib', 'nivolumab', 'dexamethasone', 'stage', 'tstage',
       'mstage', 'nstage', 'Complete therapeutic response_wind_0_45',
       'Partial therapeutic response_wind_0_45', 'Stable_wind_0_45',
       'Tumor progression_wind_0_45',
       'Complete therapeutic response_wind_45_180',
       'Partial therapeutic response_wind_45_180', 'Stable_wind_45_180',
       'Tumor progression_wind_45_180', 'biomarker_KRAS', 'biomarker_TP53',
       'biomarker_TMB', 'biomarker_PD-L1', 'biomarker_EGFR', 'biomarker_BRAF',
       'biomarker_PTEN', 'biomarker_TERT', 'biomarker_KIT', 'biomarker_PIK3CA',
       'biomarker_CDH1', 'biomarker_STK11', 'biomarker_ALK', 'CT', 'PET',
       'MRI', 'Radioisotope', 'cohort'])

In [ ]:
x_train.shape

In [ ]:
X = X[x_train.columns]

In [ ]:
event_prob = 1-xgbse_model.predict(X)
final_data = pd.concat([final_data,event_prob],axis=1)
final_data.shape

In [ ]:
final_data.to_csv(f'{output_dir}inference_final_data.csv',index=False)
# final_data = pd.read_csv(f'{output_dir}inference_final_data.csv')

In [ ]:
final_data.head()

In [ ]:
final_data[final_data['event']==0]['TTE'].describe()

In [ ]:
final_data_90['TTE'].describe()

In [ ]:
final_data['event'].unique()

In [ ]:
final_data_90 = final_data[['chai_patient_id','TTE','event',90]]
final_data_90['event_90'] = final_data_90.apply(lambda x:0 if x['TTE']>90 else x['event'],axis=1)

In [ ]:
pa = final_data_90[['chai_patient_id','event_90',90]]
pa.columns = ['chai_patient_id','actual','pred']
pa.head()

In [ ]:
final_data_90['chai_patient_id'].shape

In [ ]:
pa_q = pa.copy()
pa_q['quantile'] = pd.qcut(pa['pred'],q = 4, labels = False)
pa_q.head()

In [ ]:
probs = pd.DataFrame(columns=['quartile','min_prob','max_prob'])
for qt in [0,1,2,3]:
    probs = probs.append({'quartile':qt,
                        'min_prob':pa_q[pa_q['quantile']==qt].pred.min(),
                        'max_prob':pa_q[pa_q['quantile']==qt].pred.max()},ignore_index=True)

probs.to_csv(f'{output_dir}inference/quartile_min_max_prob.csv',index=False)
probs

In [ ]:
roc = []
for qt in [0,1,2,3]:
    actual = pa_q[pa_q['quantile']==qt].actual
    pred   = pa_q[pa_q['quantile']==qt].pred
    auc    = roc_auc_score(actual,pred)
    # print('Quartile:', qt, ' AUC = ',auc)
    roc.append(pd.DataFrame({'Quartile':[qt],'AUC':[auc]}))
               
roc = pd.concat(roc)
roc          

In [ ]:
roc_group = pd.DataFrame({'top2_quartiles':roc_auc_score(pa_q[pa_q['quantile']>1].actual, pa_q[pa_q['quantile']>1].pred),
                    'top1_quartiles':roc_auc_score(pa_q[pa_q['quantile']>2].actual, pa_q[pa_q['quantile']>2].pred),
                    'bottom2_quartiles':roc_auc_score(pa_q[pa_q['quantile']<2].actual, pa_q[pa_q['quantile']<2].pred),
                    'full_dataset':roc_auc_score(pa_q.actual, pa_q.pred)}.items())
roc_group.columns = ['Quartile','AUC']

roc = pd.concat([roc,roc_group])

probs.to_csv(f'{output_dir}inference/quartile_auc.csv',index=False)
roc

In [ ]:
f1 = pd.DataFrame(columns=['th','precision','recall'])
for th in range(0, 100, 1):
    tmp = pa_q[['chai_patient_id','actual','pred']]
    tmp['pred_class'] = np.where(tmp.pred>=th/100, 1, 0)
    tmp['actual_class'] = tmp.actual
    f1 = f1.append({'th':th, 
                    'precision':precision_score(tmp.actual_class, tmp.pred_class, average='weighted'),
                    'recall':recall_score(tmp.actual_class, tmp.pred_class, average='weighted'),
                    'f1':f1_score(tmp.actual_class, tmp.pred_class, average = 'weighted')}, ignore_index=True)

In [ ]:
prf_plot = f1[['precision','recall','f1']].plot()
fig = prf_plot.get_figure()
fig.savefig(f'{output_dir}inference/Pr_Re_F1_plot.png',bbox_inches = 'tight')

In [ ]:
pa_q.head()

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

PrecisionRecallDisplay.from_predictions(pa_q['actual'], pa_q['pred'], name="PA_pan_solid_model", plot_chance_level=True)
plt.title(f'window : 90')
plt.legend(loc=(0.4, 0.8))
plt.plot()

In [ ]:
event_prob_df = event_prob_df.drop('Unnamed: 0',axis=1)
x_test = x_test.drop('Unnamed: 0',axis=1)
final_data = final_data.drop('Unnamed: 0',axis=1)

In [ ]:
final_data[final_data['event']==1]['TTE'].hist(bins=100)

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import PrecisionRecallDisplay,ConfusionMatrixDisplay,confusion_matrix,roc_auc_score
import matplotlib.pyplot as plt
combined_test_info_df = event_prob_df.join(x_test.reset_index()).merge(final_data[['chai_patient_id','TTE','event']].reset_index(),on='index',how='inner')

# combined_test_info_df = event_prob_df.join(x_test.reset_index()).merge(final_stat_df.reset_index()[['index','chai_patient_id','label','final_selection']],on='index',how='inner')
for i in [60,90,120,150]:
    combined_test_info_df[f'final_selection_{i}'] = combined_test_info_df.apply(lambda x: 0 if x['TTE']>i else x['event'],axis=1)
    print(combined_test_info_df[f'final_selection_{i}'].value_counts())
    print(f"ROC AUC Score : {roc_auc_score(combined_test_info_df[f'final_selection_{i}'], combined_test_info_df[i])}")
    text_file.write(f'Label counts for window : {i} \n')
    text_file.write(f"{combined_test_info_df[f'final_selection_{i}'].value_counts()} \n\n")
    PrecisionRecallDisplay.from_predictions(combined_test_info_df[f'final_selection_{i}'], combined_test_info_df[i], name="PA_model", plot_chance_level=True)
    plt.title(f'window : {i}')
    plt.legend(loc=(0.5, 0.8))
    plt.savefig(f'{output_dir}/PRC_Window_{i}.png')
    plt.plot()

In [ ]:
feature_importance_df = pd.DataFrame(xgbse_model.feature_importances_.items(),columns=['feature','importance'])

In [ ]:
feature_importance_df =feature_importance_df.sort_values(by='importance',ascending=False)
combine_feature_importance_df_subset = feature_importance_df[:20]
combine_feature_importance_df_subset = combine_feature_importance_df_subset.sort_values(by='importance',ascending=True)

In [ ]:
f, ax = plt.subplots(figsize=(20,10))
plt.barh(combine_feature_importance_df_subset['feature'],combine_feature_importance_df_subset['importance'])
# plt.xticks(range(len(combine_feature_importance_df_subset['feature'])), combine_feature_importance_df_subset['average'],rotation='vertical')
plt.savefig(f'{output_dir}/importance_plot.png',bbox_inches = 'tight')
plt.show()

In [ ]:
combined_test_info_df.head()

In [ ]:
for window in [60,90,120,150]:
    combined_test_info_df[combined_test_info_df[f'final_selection_{window}']==True][window].plot.kde()
    combined_test_info_df[combined_test_info_df[f'final_selection_{window}']==False][window].plot.kde()
    plt.legend(['Event','Censored'])
    plt.title(f'Prediction window : {window}')
    plt.savefig(f'{output_dir}/pred_prob_classwise_kde_window_{window}.png',bbox_inches = 'tight')
    plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix
# combined_test_info_df = event_prob_df.join(x_test.reset_index()).merge(final_stat_df.reset_index()[['index','chai_patient_id','label','final_selection']],on='index',how='inner')
metric_list = []
for window in [60,90,120,150]:
    window_df = pd.DataFrame({'window':[window]})
    for thresold in [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
        combined_test_info_df[f'pred_{thresold}'] = combined_test_info_df[window].apply(lambda x:x>thresold)
        cm = confusion_matrix(combined_test_info_df[f'final_selection_{i}'],combined_test_info_df[f'pred_{thresold}'])
        tn,fp,fn,tp = cm[0,0],cm[0,1],cm[1,0],cm[1,1]
        p1 = tp/(tp+fp)
        r1 = tp/(tp+fn)
        f1 = 2*p1*r1/(p1+r1)
        p0 = tn/(tn+fn)
        r0 = tn/(tn+fp)
        f0 = 2*p0*r0/(p0+r0)
        window_df = pd.concat([window_df,pd.DataFrame({
            f'tn_{thresold}' : [tn],
            f'fp_{thresold}' : [fp],
            f'fn_{thresold}' : [fn],
            f'tp_{thresold}' : [tp],
            f'p1_{thresold}' : [p1],
            f'r1_{thresold}' : [r1],
            f'f1_{thresold}' : [f1],
            f'p0_{thresold}' : [p0],
            f'r0_{thresold}' : [r0],
            f'f0_{thresold}' : [f0],
        })],axis=1)
    metric_list.append(window_df)
metric_df = pd.concat(metric_list)

In [ ]:
metric_df

In [ ]:
text_file.close()

In [ ]:
import shap
x_train_sample =  x_train.sample(n=200,random_state=123).astype('float64')
explainer = shap.Explainer(xgbse_model.predict,x_train_sample)
shap_values = explainer(x_train_sample,max_evals='auto')

In [ ]:
text_file.plots.beeswarm(shap_values[:,:,0],max_display=30)
plt.savefig(f'{output_dir}/shap_plot.png',bbox_inches = 'tight')